# Abazinka dictionary crowler

___Ksenia R.___

Crowler to fill in certain columns in the final table. 

**Plan**
- get links automatically
- select and load relevant pages
- work with the files and fill in the table

Request to the page of a letter (here: $a$). 

In [1]:
import requests
page = requests.get("http://abazinka.ru/ru/letter/А?")
page 

<Response [200]>

`200` means that the page downloaded successfully: 

In [2]:
# неактуально до тех пор пока всё работает как нужно 

# page.status_code 

In [3]:
# большая выдача, неактуально 
# выдаёт содержимое страницы 

# page.content

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
# soup.find_all('a')

Then I get the parts of useful links:
- only with `russian` in its link (the word pages)
- only verbs, i.e. words ending with `-ра` which is masdar, verb dictionary entry marker

Example output: 
>/abaza/russian/а зунагIвахара  
>/abaza/russian/а сара  
>/abaza/russian/а-ахьарпара  
>/abaza/russian/а-ацыцындра  
>/abaza/russian/а-жьгара  
>/abaza/russian/а-расатара  
>/abaza/russian/а-сахьхэыра  
>/abaza/russian/а-цыбра  
>/abaza/russian/а-цыцындра  
>/abaza/russian/абабалара

In [5]:
# получаю ссылки на словарные статьи с глаголами 

link_prim = '' 
for link in soup.findAll('a'):
    new_link = (link.get('href'))
    if 'russian' in new_link:       # dictionary page 
        if new_link.endswith('ра'):       # MSD for verbs 
            if new_link not in link_prim:
                link_prim = link_prim + new_link + '\n' 
    
    #print (link.get('href'))

In [6]:
# большая выдача, работает как нужно, 
# см. выше начало выдачи  

# print(link_prim) 

Replacement of spaces to `%20` for link format, consider: [_http://abazinka.ru/abaza/russian/а%20ды-ды-дыд%20мыгIва_](http://abazinka.ru/abaza/russian/а%20ды-ды-дыд%20мыгIва)

In [7]:
#for line in link_prim: 
link_prim = link_prim.replace(' ', '%20')  # 'murmurian'

# большая выдача, работает как нужно 
# print(link_prim)

**Консультация с DH специалистом**

+ как лучше устроить цикл? 

    - ну ты можешь сохранять инфу по ходу, типа сохранять в json или типа того
    - скачала одну нужную и созранила
    - ничего не будет накапливаться, а когда надо будет, ты считаешь из файла

+ ага, то есть создавать файлы лучше, чем одним циклом всё обработать?

    + да, может чето отвалиться по ходу
    + чтобы не начинать заново
    + лучше запомниать, где ты останавливаешься

In [8]:
link_l = link_prim.split('\n')
# большая выдача, работает как нужно 
# print(link_l)

In [9]:
# проверка актуального номера 
# print(link_l[187])

In [10]:
# проверка длины массива ссылок, чтобы дальше вставить в функцию range()
print(len(link_l)) 

5691


Now i work with link lists: 
- create links and open links
- then i work only with the pages which contain number $2$ in them, i consider it as a marker of more than one meaning recorded.  
- i recorde these pages in txt files in a folder for the letter, the name of the file is a word (with `%20` instead of spaces) `.txt` format
- also i keep the list of words which are recorded `word_list` variable

In [11]:
# тут создаются ссылки, открываются, проверяются, скачиваются 

link_start = 'http://abazinka.ru'
link_full = '' 
word_list = []

for i in range(0, 5690): # 272/7 # 180 5690
    # print(str(link_l[i]))
    link_full = link_start + str(link_l[i])
    # print(link_full)
    
    dict_page = requests.get(link_full)
    # print(dict_page)
    # dict_page.encoding = 'utf-8'  # указываем правильную кодировку принудительно
    d_p_text = (dict_page.text) 
    # print(d_p_text) 
    
    if '2' in d_p_text: 
        f_name = str(link_l[i]).split('/')
        word_list.append(f_name[3])
        
        # f_name = '/Users/ksenia/Desktop/hse/MA courses/практика/a_letter/' + str(f_name[2]) + '.txt'
        f_name = 'a_letter/' + str(f_name[3]) + '.txt'
        # print(f_name) 
        
        f = open(f_name, 'w')
        f.write(d_p_text)
        f.close()
        # print(d_p_text)


**Here i create two files with:**
- simple text file with list of words `wordlist.txt`
- a first version of the table `table_words.tsv` with enumeration and headers
    - Verb ID
    - Verb (dictionary entry)
    - Root
    - Meaning of the root (if known)
    - Glossing of the verb form (if possible)
    - Sense 1
    - Example for the Sense 1 (if available)
    - Sense 2
    - Example for the Sense 2 (if available)
    - Semantic shift
    - Comments1
    - Comments2

In [15]:
# сохраню в отдельный файл слова, чтобы заново не запускать предыдущий цикл 
# print(word_list) 

fi = open('wordlist.txt', 'w')
table_1 = open('table_words.tsv', 'w')
table_1.write('Verb ID	Verb (dictionary entry)	Root	Meaning of the root (if known)	Glossing of the verb form (if possible)	Sense 1	Example for the Sense 1 (if available)	Sense 2	Example for the Sense 2 (if available)	Semantic shift	Comments1	Comments2')
num = 0

for elem in word_list:
    num = num + 1
    elem_s = str(elem).replace('%20', ' ')  # 'murmurian'
    
    fi.write(elem_s + '\n')
    table_1.write('\n' + str(num) + '\t' + elem_s + '\t' + '\t' + '\t' + '\t' + '\t' + '\t' + '\t' + '\t' + '\t' + '\t')

fi.close()
table_1.close()

In [ ]:
# Далее: обработка и записывание в таблицу 